# Schrödinger's Python 3

<img src="https://github.com/natsunoyuki/blog_posts/blob/main/images/schrodinger_eq_1d.png?raw=True" alt="drawing" width="333"/>

In my previous posts, we explored the <a href="https://github.com/natsunoyuki/blog_posts/blob/main/physics/Schrödinger's%20Python.ipynb" target="_blank">1 dimensional</a> and <a href="https://github.com/natsunoyuki/blog_posts/blob/main/physics/Schrödinger's%20Python%202.ipynb" target="_blank">2 dimensional</a> time independent <a href="https://en.wikipedia.org/wiki/Schrödinger_equation" target="_blank">Schrödinger's equation</a>. Most useful applications require solving the system in at least 2 dimensions. However, to make everything complete we also need to take a look at the 3 dimensional solver which is what we will do in this post. If you haven't yet, I recommend reading the previous 2 posts for more information about the fundamentals.

## The 3D Schrödinger Equation

As with the 1D and 2D numerical discretisation schemes, we set _ħ_ = _m_ = 1 in order to simplify things. Also, we scale the energies _V_ and _E_ by a factor of 2. The 3 dimensional discretized equation takes on the form below.

<img src="https://github.com/natsunoyuki/blog_posts/blob/main/images/schrodinger_eq_3d.png?raw=True" alt="drawing" width="700"/>

Note that _dx_, _dy_ and _dz_ are the numerical derivative step sizes for _x_, _y_ and _z_ respectively. However in such a discretized form, the wave function __Ψ__ becomes a 3 dimensional matrix with 3 indices _n_, _m_ and _l_. From a numerical eigenvalue solver standpoint, this is unacceptable as we want to solve for the system’s eigen energies and eigen wave functions which requires __Ψ__ to be in the form of a 1 dimensional vector. As with the 2 dimensional case, we use Kronecker products to combine the 3 one dimensional Hilbert spaces for _x_, _y_ and _z_ together to form a single three dimensional Hilbert space. The code used to generate the 3 dimensional Hamiltonian and solve for its eigenvalues is provided below.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse, linalg
from scipy.sparse import linalg as sla
 
def schrodinger3D(xmin, xmax, Nx, 
                  ymin, ymax, Ny, 
                  zmin, zmax, Nz, 
                  Vfun3D, params, neigs, E0=0.0, findpsi=False):
    """
    This function solves the 3 dimensional Schrodinger equation numerically.
    """
    x = np.linspace(xmin, xmax, Nx)  
    dx = x[1] - x[0]  
    y = np.linspace(ymin, ymax, Ny)
    dy = y[1] - y[0]
    z = np.linspace(zmin, zmax, Nz)
    dz = z[1] - z[0]

    V = Vfun3D(x, y, z, params)

    # create the 3D Hamiltonian matrix
    Hx = create_hamiltonian(Nx, dx)
    Hy = create_hamiltonian(Ny, dy)
    Hz = create_hamiltonian(Nz, dz)
    
    Ix = sparse.eye(Nx)
    Iy = sparse.eye(Ny)
    Iz = sparse.eye(Nz)
    
    # Combine the 3 individual 1 dimensional Hamiltonians
    # using Kronecker products
    Hxy = sparse.kron(Iy, Hx) + sparse.kron(Hy, Ix)
    Ixy = sparse.kron(Iy, Ix)
    H = sparse.kron(Iz, Hxy) + sparse.kron(Hz, Ixy)
    
    # Convert to lil form and add potential energy function
    H = H.tolil()
    for i in range(Nx * Ny * Nz):
        H[i, i] = H[i, i] + V[i]    

    # convert to csc form and solve the eigenvalue problem
    H = H.tocsc()  
    [evl, evt] = sla.eigs(H, k=neigs, sigma=E0)
            
    if findpsi == False:
        return evl
    else: 
        return evl, evt, x, y, z
    
def create_hamiltonian(Nx, dx):
    """
    This function creates a 1 dimensional numerical Hamiltonian.
    """
    H = sparse.eye(Nx, Nx, format='lil') * 2
    for i in range(Nx - 1):
        H[i, i + 1] = -1
        H[i + 1, i] = -1
    H = H / (dx ** 2)  
    return H

Note that setting up the Hamiltonian matrix using the Python code above results in <a href="https://en.wikipedia.org/wiki/Dirichlet_boundary_condition" target="_blank">Dirichlet boundary conditions</a> being intrinsically applied to the numerical system. This should work for most situations in general, but might break down for situations which require other forms of boundary conditions such as <a href="https://en.wikipedia.org/wiki/Periodic_boundary_conditions" target="_blank">periodic boundary conditions</a>.

## 3D Quantum Harmonic Oscillator

As plotting three dimensional solutions are not easy (we will need 4 spatial dimensions to visualize everything properly), and the visualizations are hard to understand for the matter, this time round I will simply solve for the eigenenergies only. Once again I use the quantum harmonic oscillator as the test system. The theoretical eigenenergies of the 3 dimensional quantum harmonic oscillator are given by: <a href="https://en.wikipedia.org/wiki/Quantum_harmonic_oscillator" target="_blank">_E_ = _ħω_(_n_ + 3/2)</a>. However, note that we set _ħ_ = _ω_ = 1, and we scale the energies by a factor of 2 during the construction of the numerical Hamiltonian so the theoretical energies take the new form: _E_ = 2<i>n</i> + 3 instead.

In [2]:
def sho_eigenenergies(xmin = -5, xmax = 5, Nx = 50, ymin = -5, ymax = 5, Ny = 50, 
                      zmin = -5, zmax = 5, Nz = 50, params = [1, 1, 1], neigs = 10, E0 = 0):
    """
    This function calculates the quantum simple harmonic oscillator eigenenergies.
    Theoretically, the eigenenergies are given by: E = hw(n + 3/2), n = nx + ny + nz.
    However, as we set h = w = 1, and we scale the energies during the Hamiltonian creation
    by 2, the theoretical eigenenergies are given by: E = 2n + 3.
    """
    def Vfun(X, Y, Z, params):
        Nx = len(X)
        Ny = len(Y)
        Nz = len(Z)
        M = Nx * Ny * Nz
        V = np.zeros(M)
        vindex = 0
        for i in range(Nx):
            for j in range(Ny):
                for k in range(Nz):
                    V[vindex] = params[0]*X[i] ** 2 + params[1]*Y[j] ** 2 + params[2]*Z[k] ** 2
                    vindex = vindex + 1
        return V
    
    # Only eigenvalues will be returned!
    evl = schrodinger3D(xmin, xmax, Nx, ymin, ymax, Ny, zmin, zmax, Nz, Vfun, params, neigs, E0, False)
    
    indices = np.argsort(evl)
    print("Energy eigenvalues:")
    for i,j in enumerate(evl[indices]):
        print("{}: {:.2f}".format(i + 1, np.real(j)))
        
    return sorted(evl)

Running the solver for the 3 dimensional quantum harmonic oscillator above prints out the following first ten eigenenergies:

In [3]:
evl = sho_eigenenergies(xmin = -5, xmax = 5, Nx = 30, ymin = -5, ymax = 5, Ny = 30, 
                        zmin = -5, zmax = 5, Nz = 30, params = [1, 1, 1], neigs = 10, E0 = 0)

Energy eigenvalues:
1: 2.98
2: 4.95
3: 4.95
4: 4.95
5: 6.89
6: 6.89
7: 6.89
8: 6.92
9: 6.92
10: 6.92


These energies agree very closely to the actual theoretical values of 3, 5, and 7. The eigenvalues also show the proper number of degeneracies for 5 and 7. This shows that the solver is working nicely!

Unfortunately when more dimensions are added to the system, the amount of memory required to process the entire calculation increases very quickly. Powerful computers will be required to perform calculations within large 3D grids.

***
## My recommended books to learn more about Quantum Mechanics:
(I am in no way related or affiliated to the links provided!)

* <a href="https://www.amazon.com/Introduction-Quantum-Mechanics-David-Griffiths/dp/1107179866" target="_blank">Introduction to Quantum Mechanics</a>, D. J. Griffiths - this book is a great book for those who do not have an advanced physics background. Also this book has a picture of a cat on the cover!
* <a href="https://www.amazon.com/Quantum-Mechanics-2nd-B-H-Bransden/dp/0582356911" target="_blank">Quantum Mechanics</a>, B. H. Bransden and C. J. Joachain - for the more advanced people, I would recommend this introductory book which is more thorough and detailed mathematically. Unfortunately no cats on the cover.